In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import sys

from importlib import reload


import src.cathay_db as db
import src.utils as ut
import src.financial_statement as fs

reload(ut)
reload(fs)
import matplotlib.pyplot as plt
import datetime
import os

# set max display rows
pd.set_option("display.max_rows", 200)

# set max display columns
pd.set_option("display.max_columns", 100)

# Set the float format to display without scientific notation
pd.options.display.float_format = '{:.4f}'.format

# 01 Read data from xlsx file

In [ ]:
df_delisted = ut.concatenate_excel_sheets('./data/tej_財報_2000_2023_下市.xlsx')
df_listed_IFRS = ut.concatenate_excel_sheets('./data/tej_IFRS_2005_2023_上市.xlsx')
df_listed_Finance_DB = ut.concatenate_excel_sheets('./data/tej_FinanceDB_2000_2019_上市.xlsx')
df_delisted_lsd = ut.concatenate_excel_sheets('./data/tej_長短期借款_2000_2023_下市.xlsx')
# dataframe with cum means cumulative data
df_listed_IFRS_cum = ut.concatenate_excel_sheets('./data/tej_IFRS_cum_2005_2023_上市.xlsx')

df_delisted_lsd = ut.tej_data_clean_up(df_delisted_lsd)
df_delisted_lsd.drop_duplicates(inplace=True)

df_delisted = ut.tej_data_clean_up(df_delisted)
df_listed_IFRS = ut.tej_data_clean_up(df_listed_IFRS)
df_listed_Finance_DB = ut.tej_data_clean_up(df_listed_Finance_DB)
df_listed_IFRS_cum = ut.tej_data_clean_up(df_listed_IFRS_cum)

# drop duplicates
df_delisted.drop_duplicates(inplace=True)
df_listed_IFRS.drop_duplicates(inplace=True)
df_listed_Finance_DB.drop_duplicates(inplace=True)
df_listed_IFRS_cum.drop_duplicates(inplace=True)

# remove the "Q" and "M" and any leading and trailing spaces in column names
df_delisted.columns = [col[1:].strip() if col[0] == 'Q' else col.strip() for col in df_delisted.columns]
df_listed_IFRS.columns = [col.strip() for col in df_listed_IFRS.columns]
df_listed_Finance_DB.columns = [col[1:].strip() if col[0] == 'M' else col.strip() for col in df_listed_Finance_DB.columns]

# to feather
df_delisted.reset_index(drop=True).to_feather('./data/tej_財報_2000_2023_下市.feather')
df_listed_IFRS.reset_index(drop=True).to_feather('./data/tej_IFRS_2005_2023_上市.feather')
df_listed_Finance_DB.reset_index(drop=True).to_feather('./data/tej_FinanceDB_2000_2019_上市.feather')
df_listed_IFRS_cum.reset_index(drop=True).to_feather('./data/tej_IFRS_cum_2005_2023_上市.feather')

In [4]:
df_delisted_lsd = ut.concatenate_excel_sheets('./data/tej_長短期借款_2000_2023_下市.xlsx')
df_delisted_lsd = ut.tej_data_clean_up(df_delisted_lsd)
df_delisted_lsd.drop_duplicates(inplace=True)

In [6]:
df_delisted_lsd.reset_index(drop=True).to_feather('./data/tej_長短期借款_2000_2023_下市.feather')

In [24]:
# from feather
df_delisted = pd.read_feather('./data/tej_財報_2000_2023_下市.feather')
df_listed_IFRS = pd.read_feather('./data/tej_IFRS_2005_2023_上市.feather')
df_listed_Finance_DB = pd.read_feather('./data/tej_FinanceDB_2000_2019_上市.feather')
df_listed_IFRS_cum = pd.read_feather('./data/tej_IFRS_cum_2005_2023_上市.feather')
df_delisted_lsd = pd.read_feather('./data/tej_長短期借款_2000_2023_下市.feather')

# 02 Merge data from different sources

In [25]:
df_delisted = df_delisted.merge(df_delisted_lsd[['股票代號','年月','長短期借款']], on=['股票代號', '年月'], how='left')
# create '長短期借款', '財報發布日' for the listed companies
df_listed_Finance_DB['長短期借款'] = np.nan
df_listed_Finance_DB['財報發布日'] = np.nan
# create '財報發布日' for the delisted companies
df_delisted['財報發布日'] = np.nan

# create 'sequence' column for the different dataframes, the smaller the number, the more priority
df_listed_Finance_DB['sequence'] = 2
df_listed_IFRS_cum['sequence'] = 1
df_delisted['sequence'] = 3

# create a new dataframe with the columns of these 3 dataframes
df_delisted_columns = pd.DataFrame(df_delisted.columns, columns=['delisted'])
df_listed_IFRS_cum_columns = pd.DataFrame(df_listed_IFRS_cum.columns, columns=['IFRS'])
df_listed_Finance_DB_columns = pd.DataFrame(df_listed_Finance_DB.columns, columns=['Finance_DB'])

# merge all df by the column names
df_all_columns = pd.merge(df_delisted_columns, df_listed_IFRS_cum_columns, how='outer', left_on='delisted', right_on='IFRS')
df_all_columns = pd.merge(df_all_columns, df_listed_Finance_DB_columns, how='outer', left_on='delisted', right_on='Finance_DB')

df_all_columns = df_all_columns.dropna(how = 'any')

# pick only the columns from the delisted dataframe
common_column = df_all_columns['delisted'].tolist()

df_fs_all = pd.DataFrame([])
# concat the all df
df_fs_all = pd.concat([df_fs_all, df_listed_IFRS_cum[common_column]],axis=0)
df_fs_all = pd.concat([df_fs_all, df_listed_Finance_DB[common_column]],axis=0)
df_fs_all = pd.concat([df_fs_all, df_delisted[common_column]],axis=0)

# get the min sequence of each stock at each year
df_seq = df_fs_all.groupby(['股票代號','年月'])['sequence'].min().reset_index().copy()
df_fs_merge = pd.merge(df_seq, df_fs_all, how='left', on=['股票代號','年月','sequence'])

df_fs_merge.sort_values(['股票代號','年月'], inplace=True)
# filter out the data that '股票代號' length is not 4
df_fs_merge = df_fs_merge[df_fs_merge['股票代號'].str.len() == 4]
df_fs_merge.reset_index(drop=True, inplace=True)
df_fs_merge.to_feather('./data/df_tej_fs_merge.feather')

In [26]:
df_fs_merge = pd.read_feather('./data/df_tej_fs_merge.feather')

# 03 Merge the OCF and NI data to dataframe

In [27]:
df_slice_cum = ut.concatenate_excel_sheets('./data/tej_IFRS_slice_cum_2000_2023_All.xlsx')

# because these columns are cumulative, so we need to delete them
del df_slice_cum['來自營運之現金流量']
del df_slice_cum['繼續營業單位損益']

# merge the single season '來自營運之現金流量' from df_fs_merge
df_slice_cum = pd.merge(df_slice_cum, df_fs_merge[['公司','年月','來自營運之現金流量']], how='left', on=['公司','年月'])
df_slice_cum = ut.tej_data_clean_up(df_slice_cum)

# then handle the cumulative '繼續營業單位損益', named as 'ni'
df_slice_ni_cum = ut.concatenate_excel_sheets('./data/tej_IFRS_ni_slice_cum_2000_2023_All.xlsx')
df_slice_ni_cum = ut.tej_data_clean_up(df_slice_ni_cum)
df_slice_ni_cum.sort_values(['公司','年月'], inplace=True)
df_slice_ni_cum['last_date'] = df_slice_ni_cum.groupby('股票名稱')['年月'].shift(1)

# cumulative is yearly reset, so replace the last_date to np.nan if the last_date is not the same year with 年月, ex: 2019-12-31, 2020-03-31
df_slice_ni_cum.loc[df_slice_ni_cum['last_date'].astype('str').str[:4] != df_slice_ni_cum['年月'].astype('str').str[:4], 'last_date'] = np.nan
df_slice_ni_cum = df_slice_ni_cum.merge(df_slice_ni_cum[['公司','年月','歸屬母公司淨利（損）']], how='left', left_on=['公司','last_date'], right_on=['公司','年月'], suffixes=('', '_y'))
df_slice_ni_cum['ni'] = df_slice_ni_cum['歸屬母公司淨利（損）']  - df_slice_ni_cum['歸屬母公司淨利（損）_y']
df_slice_ni_cum['ni'] = np.where(df_slice_ni_cum['ni'].isna(), df_slice_ni_cum['歸屬母公司淨利（損）'], df_slice_ni_cum['ni'])

df_slice_cum = pd.merge(df_slice_cum, df_slice_ni_cum[['公司','年月','ni']], how='left', on=['公司','年月'])
df_slice_cum.sort_values(['公司','年月'], inplace=True)
df_slice_cum.reset_index(drop=True, inplace=True)
df_slice_cum.to_feather('./data/df_tej_fs_slice_cum.feather')

# 04 Merge the OPM to dataframe

In [28]:
df_slice_cum = pd.read_feather('./data/df_tej_fs_slice_cum.feather')

df_tej_ifrs_opm = ut.concatenate_excel_sheets('./data/tej_ifrs_opm_2000_2023_All.xlsx')
df_tej_new_finance_opm = ut.concatenate_excel_sheets('./data/tej_new_finance_opm_2000_2023_All.xlsx')
df_tej_finance_opm = ut.concatenate_excel_sheets('./data/tej_finance_opm_2000_2023_All.xlsx')

# drop the duplicate rows
df_tej_ifrs_opm.drop_duplicates(inplace=True)
df_tej_new_finance_opm.drop_duplicates(inplace=True)
df_tej_finance_opm.drop_duplicates(inplace=True)

df_tej_ifrs_opm = df_tej_ifrs_opm[['公司', '年月', '季別', '營業收入淨額', '營業利益']]
df_tej_new_finance_opm = df_tej_new_finance_opm[['公司', '年月', '季別', 'Q營業收入淨額', 'Q營業利益']]
df_tej_finance_opm = df_tej_finance_opm[['公司', '年月', '季別', 'M營業收入淨額', 'M營業利益']]

# rename the columns for concat
df_tej_ifrs_opm.columns = ['公司', '年月', '季別', '營業收入淨額', '營業利益']
df_tej_new_finance_opm.columns = ['公司', '年月', '季別', '營業收入淨額', '營業利益']
df_tej_finance_opm.columns = ['公司', '年月', '季別', '營業收入淨額', '營業利益']

df_tej_ifrs_opm['sequence'] = 2
df_tej_new_finance_opm['sequence'] = 1
df_tej_finance_opm['sequence'] = 3

# concat the 3 dataframes
df_tej_opm_all = pd.concat([df_tej_ifrs_opm, df_tej_new_finance_opm, df_tej_finance_opm], axis=0)

# get the min sequence of each stock at each year
df_seq = df_tej_opm_all.groupby(['公司','年月'])['sequence'].min().reset_index().copy()
df_opm_merge = pd.merge(df_seq, df_tej_opm_all, how='left', on=['公司','年月','sequence'])

df_opm_merge.sort_values(['公司','年月'], inplace=True)
# filter out the data that '公司' length is not 4
# df_opm_merge = df_opm_merge[df_opm_merge['公司'].str.len() == 4]
df_opm_merge.reset_index(drop=True, inplace=True)

In [34]:
df_slice_cum = df_slice_cum.merge(df_opm_merge[['公司','年月','營業收入淨額','營業利益']], how='left', on=['公司','年月'])

In [35]:
df_slice_cum.to_feather('./data/df_tej_fs_20230712.feather')

In [ ]:
# # merge two dataframes by '公司', '年月'
# df_tej_ifrs_opm.columns = ['公司', '年月', '季別', '營業收入淨額', '營業利益','sequence']
# df_tej_new_finance_opm.columns = ['公司', '年月', '季別', 'Q營業收入淨額', 'Q營業利益','sequence']
# df_tej_finance_opm.columns = ['公司', '年月', '季別', 'M營業收入淨額', 'M營業利益','sequence']

# df_tej_opm = pd.merge(df_tej_ifrs_opm[['公司', '年月','營業收入淨額','營業利益']], df_tej_new_finance_opm[['公司', '年月','Q營業收入淨額','Q營業利益']], how='outer', on=['公司','年月'], suffixes=('_ifrs', '_new_finance'))
# df_tej_opm = pd.merge(df_tej_opm, df_tej_finance_opm[['公司', '年月','M營業收入淨額','M營業利益']], how='outer', on=['公司','年月'], suffixes=('_new_finance', '_finance'))
# df_tej_opm[df_tej_opm['公司'] == '2330 台積電'].sort_values('年月')

In [ ]:
df_opm_merge['OPM'] = df_opm_merge['營業利益'] / df_opm_merge['營業收入淨額']

In [ ]:
df_slice_ni_cum = ut.concatenate_excel_sheets('./data/tej_IFRS_ni_slice_cum_2000_2023_All.xlsx')

In [ ]:
df_slice_cum